In [234]:
import torch
from utils.plotter import MetricsPlotter
from utils.logger import Logger
from utils.utils import set_settings
from data import experiment
from utils.config import get_config
op_name = ['con1', 'con3', 'max3', 'input', 'output', 'Global']
args = get_config()
set_settings(args)
args.train_size = 400
# logger plotter
exper_detail = f"Dataset : {args.dataset.upper()}, Model : {args.model}, Train_size : {args.train_size}, Bs : {args.bs}, Rank : {args.rank}, "
exper_detail += f"Train Device : {args.train_device} "
log_filename = f'Model_{args.model}_T{args.train_size}_R{args.rank}'
log = Logger(log_filename, exper_detail, args)
plotter = MetricsPlotter(log_filename, args)
args.log = log
log(str(args.__dict__))

|2024-10-08 17:18:53| {
     'att': 0, 'bs': 64, 'classification': False, 'cross': 0,
     'dataset': cpu, 'debug': False, 'decay': 0.0001, 'density': 0.8,
     'device': cpu, 'device_name': core-i7-7820x, 'epochs': 1000, 'eval_device': desktop-cpu-core-i9-13900k-fp32,
     'experiment': False, 'graph_encoder': gat, 'heads': 8, 'inductive': False,
     'llm': 1, 'logger': None, 'loss_func': L1Loss, 'lr': 0.001,
     'model': ours, 'op_encoder': embed, 'optim': AdamW, 'order': 4,
     'path': ./datasets/, 'patience': 100, 'program_test': False, 'rank': 100,
     'record': True, 'retrain': False, 'rounds': 5, 'seed': 0,
     'train_device': desktop-cpu-core-i7-7820x-fp32, 'train_size': 400, 'verbose': 0, 'visualize': True,
}
|2024-10-08 17:18:53| {'classification': False, 'visualize': True, 'inductive': False, 'bs': 64, 'lr': 0.001, 'decay': 0.0001, 'loss_func': 'L1Loss', 'optim': 'AdamW', 'path': './datasets/', 'dataset': 'cpu', 'train_size': 400, 'density': 0.8, 'logger': 'None', 'mode

In [235]:
from train_model import Model
from data import DataModule

# Initialize
exper = experiment(args)
datamodule = DataModule(exper, args)
model = Model(datamodule, args)

desktop-cpu-core-i7-7820x-fp32.pickle
|2024-10-08 17:18:53| Train_length : 400 Valid_length : 1528 Test_length : 13356 Max_value : 0.01
加载存储数据 tensor([[0.6143, 0.2500, 0.5000, 0.2750, 0.3750]])


In [236]:
model.load_state_dict(torch.load(f'checkpoints/ours/Model_ours_cpu_S{args.train_size}_R100_round_0.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [237]:
from tqdm import *
idx = 96
sample_input = datamodule.train_loader.dataset[idx]
graph, features, values = sample_input
features = features.reshape(1, -1)
print(graph, features)
from time import time 
t1 = time()
nums = 1000
for i in trange(nums):
    model(graph, features)
t2 = time()
print((t2 - t1) / nums)

Graph(num_nodes=9, num_edges=41,
      ndata_schemes={}
      edata_schemes={}) tensor([[5, 3, 0, 1, 0, 0, 2, 0, 4]])


100%|██████████| 1000/1000 [00:02<00:00, 385.43it/s]

0.002595877170562744


In [265]:
import random
def get_loss(model, sample_input):
    import torch
    
    graph, features, values = sample_input
    features = features.reshape(1, -1).long()
    
    # 清除模型中所有参数的梯度
    model.zero_grad()
    
    # 确保 info_embeds 和 graph_embeds 的梯度可以计算
    batch_size = features.shape[0]
    info_embeds = model.model.llms(batch_size)
    
    # 调用 get_node_importance，获取重要性
    graph_embeds, feats = get_node_importance(model, features)
    
    # 确保在前向传播之前设置 requires_grad=True
    info_embeds.requires_grad_(True)
    feats.requires_grad_(True)  # 注意这里对 feats 而不是整个 tuple 调用 requires_grad_
    
    # 在非叶子张量上保留梯度
    info_embeds.retain_grad()
    # feats.retain_grad()
    
    # 组合嵌入，进行前向传播
    embeds = torch.cat([graph_embeds, info_embeds], dim=1)
    output = model.model.predictor(embeds)
    
    # 使用人为定义的损失函数，比如对输出的简单加权平均
    loss = output.mean()  # 简单的均值作为损失
    loss.backward(retain_graph=True)
    return feats

def get_node_importance(model, features):
    features = features.reshape(1, -1).long()
    g, feats = graph, model.model.graph_encoder.dnn_embedding(features).reshape(features.shape[0] * 9, -1)
    attention_weights = []
    all_model = zip(model.model.graph_encoder.layers, model.model.graph_encoder.norms, model.model.graph_encoder.acts)
    for i, (layer, norm, act) in enumerate(all_model):
        # 获取 GAT 的注意力权重
        feats, attn = layer(g, feats, get_attention=True)
        feats = feats.mean(dim=1)  # 聚合多个头的输出
        feats = norm(feats)
        feats = act(feats)
        attention_weights.append(attn)
    
    batch_sizes = torch.as_tensor(g.batch_num_nodes()).to(model.model.args.device)  # 每个图的节点数
    first_nodes_idx = torch.cumsum(torch.cat((torch.tensor([0]).to(model.model.args.device), batch_sizes[:-1])), dim=0)  # 使用 torch.cat 来连接 Tensor
    first_node_features = feats[first_nodes_idx]
    
    # 使 feats 可以保留梯度
    feats.retain_grad()
    
    return first_node_features, feats

# 使用函数计算每个节点对全局节点的重要性
idx = random.randint(0, len(datamodule.train_loader.dataset))
sample_input = datamodule.train_loader.dataset[idx]
graph, features, values = sample_input

# 获取 feats 和前向传播的结果
# _, feats = get_node_importance(model, sample_input=datamodule.train_loader.dataset[idx])

# 计算损失并反向传播
print(datamodule.train_set.x[idx])
print(features)
feats = get_loss(model, sample_input=datamodule.train_loader.dataset[idx])
for i in range(len(feats)):
    print(f"key-{features[i]}--{op_name[features[i]]:6s} -- {torch.sum(feats.grad[i]):.4f}")

[2 0 3 0 0 3]
tensor([5, 3, 0, 0, 1, 0, 0, 1, 4])
key-5--Global -- -0.0331
key-3--input  -- 0.0000
key-0--con1   -- 0.0000
key-0--con1   -- 0.0000
key-1--con3   -- 0.0000
key-0--con1   -- 0.0000
key-0--con1   -- 0.0000
key-1--con3   -- 0.0000
key-4--output -- 0.0000


In [266]:
feats.grad[i].sum()

tensor(0.)

In [267]:
graph.edges()

(tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 4, 4, 5, 6, 7, 7, 7, 7,
         8, 8, 8]),
 tensor([0, 1, 2, 4, 7, 8, 0, 1, 2, 0, 1, 2, 4, 3, 0, 2, 4, 7, 5, 6, 0, 4, 7, 8,
         0, 7, 8]))

In [268]:
all_grad = [[] for _ in range(6)]
for i in range(len(datamodule.train_loader.dataset)):
    sample_input = datamodule.train_loader.dataset[i]
    graph, features, values = sample_input
    print(datamodule.train_set.x[i], features)
    feats = get_loss(model, sample_input=sample_input)
    for j in range(len(feats)):
        print(f"key-{features[j]}--{op_name[features[j]]:6s} -- {torch.sum(feats.grad[j]):.4f}")
        all_grad[features[j]].append(torch.sum(feats.grad[j]).item()) 
    # print(all_grad)

[0 2 3 1 2 3] tensor([5, 3, 0, 0, 0, 0, 0, 1, 4])
key-5--Global -- 0.0309
key-3--input  -- 0.0000
key-0--con1   -- 0.0000
key-0--con1   -- 0.0000
key-0--con1   -- 0.0000
key-0--con1   -- 0.0000
key-0--con1   -- 0.0000
key-1--con3   -- 0.0000
key-4--output -- 0.0000
[0 3 3 1 0 4] tensor([5, 3, 0, 1, 0, 0, 0, 2, 4])
key-5--Global -- -0.0362
key-3--input  -- 0.0000
key-0--con1   -- 0.0000
key-1--con3   -- 0.0000
key-0--con1   -- 0.0000
key-0--con1   -- 0.0000
key-0--con1   -- 0.0000
key-2--max3   -- 0.0000
key-4--output -- 0.0000
[4 0 0 2 3 2] tensor([5, 3, 2, 0, 0, 0, 1, 0, 4])
key-5--Global -- -0.0335
key-3--input  -- 0.0000
key-2--max3   -- 0.0000
key-0--con1   -- 0.0000
key-0--con1   -- 0.0000
key-0--con1   -- 0.0000
key-1--con3   -- 0.0000
key-0--con1   -- 0.0000
key-4--output -- 0.0000
[1 2 0 3 0 1] tensor([5, 3, 0, 0, 0, 1, 0, 0, 4])
key-5--Global -- 0.0350
key-3--input  -- 0.0000
key-0--con1   -- 0.0000
key-0--con1   -- 0.0000
key-0--con1   -- 0.0000
key-1--con3   -- 0.0000
key-0-

In [269]:
import numpy as np
for i in range(len(all_grad)):
    print(f"key-{i}--{op_name[i]:6s}: {np.mean(all_grad[i]):.50f}")

key-0--con1  : 0.00000000000000000000000000000000000000000000000000
key-1--con3  : 0.00000000000000000000000000000000000000000000000000
key-2--max3  : 0.00000000000000000000000000000000000000000000000000
key-3--input : 0.00000000000000000000000000000000000000000000000000
key-4--output: 0.00000000000000000000000000000000000000000000000000
key-5--Global: -0.02638090909342281562000032124615245265886187553406
